In [1]:
## import autosklearn.classification
from pipeline import *

# pipeline_year(10)

df_coaches = pd.read_csv('dataset/cleaned/coaches.csv')
l = []
for i in range(2, 11):
    l.append(pipeline_year(i, False))
print(l)


c:\Users\ricar\OneDrive\Ambiente de Trabalho\ac\funcs\statistical_analysis.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_coaches['win_percentage_coach'] = (df_coaches['won'] + df_coaches['post_wins']) / (df_coaches['won'] +
c:\Users\ricar\OneDrive\Ambiente de Trabalho\ac\pipeline.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['confID'] = train['confID'].replace(['EA', 'WE'], [0, 1])
c:\Users\ricar\OneDrive\Ambiente de Trabalho\ac\pipeline.py:126: SettingWithCopyWarning: 
A value is t

[62.5, 62.5, 62.5, 62.5, 87.5, 87.5, 62.5, 100.0, 87.5]


c:\Users\ricar\OneDrive\Ambiente de Trabalho\ac\pipeline.py:140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['predictions'] = predictions
